# SOPHY Database
### Southern Ocean Phytoplankton Database
- Made by Ayush Nag and Hannah-Joy Warren (add others)
- UW Oceanography (add other credits)

In [1]:
import sqlite3
import pandas as pd
import pyworms
from pandas import DataFrame
import cartopy.feature
import matplotlib.pyplot as plt
import scipy.io as sio
from shapely.geometry import Polygon, Point, MultiPoint, LineString
import cartopy.crs as ccrs
import fiona
import numpy as np
import shapely
import matplotlib.path as mpath
import geopandas as gpd
import shapely.validation
import pyproj
import alphashape
from shapely.ops import transform
from geolabel import GeoLabel

In [2]:
con = sqlite3.connect("sophy.db")
cur = con.cursor()